In [58]:
import pandas as pd
csv_path = '../csv/BTC/KRW_BTC_day.csv'
df = pd.read_csv(csv_path)

df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])
df.set_index('Unnamed: 0', inplace=True)
df.columns = df.columns.str.capitalize()

In [59]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

def RSI(values, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(values).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class RsiCross(Strategy):
    lower_bound = 30
    upper_bound = 70
    lookback_period = 14

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.lookback_period)
        self.is_buy = False
        self.is_sell = False

    def next(self):
        if self.position and (self.rsi[-1] > 30 and self.rsi[-1] < 70):
            self.position.close()
        if crossover(self.lower_bound, self.rsi):
            self.position.close()
            self.buy()
        elif crossover(self.rsi, self.upper_bound):
            self.position.close()
            self.sell()
bt = Backtest(df, RsiCross, cash=1000000000, commission=0.002)
stats = bt.run()
stats



Start                     2023-05-02 09:00:00
End                       2024-05-23 09:00:00
Duration                    387 days 00:00:00
Exposure Time [%]                   31.185567
Equity Final [$]                  447300596.0
Equity Peak [$]                  1070901606.0
Return [%]                          -55.26994
Buy & Hold Return [%]              147.103242
Return (Ann.) [%]                  -53.085039
Volatility (Ann.) [%]               20.403233
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -69.097106
Avg. Drawdown [%]                  -15.091214
Max. Drawdown Duration      268 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                   15
Win Rate [%]                        73.333333
Best Trade [%]                       6.723164
Worst Trade [%]                    -52.261119
Avg. Trade [%]                    

In [57]:
stats['_trades']



,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,26,25,26,38286420.0,38229000.0,-1492920.0,-0.001500,2023-05-01 21:03:00,2023-05-01 21:06:00,0 days 00:03:00
1,26,88,90,38184216.0,38099000.0,-2215616.0,-0.002232,2023-05-02 00:12:00,2023-05-02 00:18:00,0 days 00:06:00
2,26,91,100,38116080.0,37912000.0,-5306080.0,-0.005354,2023-05-02 00:21:00,2023-05-02 00:48:00,0 days 00:27:00
3,26,170,172,37664178.0,37797000.0,3453372.0,0.003526,2023-05-02 04:18:00,2023-05-02 04:24:00,0 days 00:06:00
4,26,203,205,37453758.0,37436000.0,-461708.0,-0.000474,2023-05-02 05:57:00,2023-05-02 06:03:00,0 days 00:06:00
...,...,...,...,...,...,...,...,...,...,...
1797,1,96547,96549,49181166.0,49102000.0,-79166.0,-0.001610,2023-11-19 07:21:00,2023-11-19 07:27:00,0 days 00:06:00
1798,-1,96702,96703,49138526.0,49235000.0,-96474.0,-0.001963,2023-11-19 15:06:00,2023-11-19 15:09:00,0 days 00:03:00
1799,-1,96704,96705,49150502.0,49220000.0,-69498.0,-0.001414,2023-11-19 15:12:00,2023-11-19 15:15:00,0 days 00:03:00
1800,1,96831,96833,49101006.0,49040000.0,-61006.0,-0.001242,2023-11-19 21:33:00,2023-11-19 21:39:00,0 days 00:06:00
